In [30]:
import torch
import pandas as pd
from torchvision import datasets
from sklearn.preprocessing import StandardScaler

In [31]:
dataset = pd.read_csv('Housing.csv')
x = dataset.iloc[:, [1, 2, 3, 4, 10]].values
y = dataset.iloc[:, 0].values

In [32]:
x[0:10]

array([[ 7420,     4,     2,     3,     2],
       [ 8960,     4,     4,     4,     3],
       [ 9960,     3,     2,     2,     2],
       [ 7500,     4,     2,     2,     3],
       [ 7420,     4,     1,     2,     2],
       [ 7500,     3,     3,     1,     2],
       [ 8580,     4,     3,     4,     2],
       [16200,     5,     3,     2,     0],
       [ 8100,     4,     1,     2,     2],
       [ 5750,     3,     2,     4,     1]], dtype=int64)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=0)

In [34]:
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

In [35]:
t_c = torch.tensor(Y_train)
t_u = torch.tensor(X_train)

In [36]:
t_u.size()

torch.Size([436, 5])

In [37]:
def model(t_u, w1, w2, w3, w4, w5, b):
    return w1*t_u[0, :] + w2*t_u[1, :] + w3*t_u[2, :] + w4*t_u[3, :] + w5*t_u[4, :] + b

In [38]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [39]:
import torch.optim as optim
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional',
 '_multi_tensor',
 'lr_scheduler',
 'swa_utils']

In [40]:
t_c.size()

torch.Size([436])

In [50]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0],
    requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

t_p = model(t_un, *params)
loss = loss_fn(t_p, t_c)

optimizer.zero_grad()
loss.backward()
optimizer.step()

params

RuntimeError: The size of tensor a (5) must match the size of tensor b (436) at non-singleton dimension 0

In [44]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p[1], t_c)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [49]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 25988971907827.246094
Epoch 1000, Loss 25517161658948.402344
Epoch 1500, Loss 25055313800788.347656
Epoch 2000, Loss 24603218110591.546875
Epoch 2500, Loss 24160668840619.394531
Epoch 3000, Loss 23727463800941.496094
Epoch 3500, Loss 23303405180293.726562
Epoch 4000, Loss 22888300826652.640625
Epoch 4500, Loss 22481962046700.839844
Epoch 5000, Loss 22084200354238.597656


tensor([-58655.1016, -58655.1016, -58655.1016, -58655.1016,   2380.8416,
        453161.3750], requires_grad=True)